## Linear Programming using CBC backend MIP solver
### Mixed Integer Programming

The goal is to maximize the power of the army with the following resources: 183000 food, 90512 wood, 80150 gold.

Unit 	🌾Food 	🪵Wood 	🪙Gold 	💪Attack 	❤️Health
🗡️Swordsman 	60 	20 	0 	6 	70
🛡️Man-at-arms 	100 	0 	20 	12 	155
🏹Bowman 	30 	50 	0 	5 	70
❌Crossbowman 	80 	0 	40 	12 	80
🔫Handcannoneer 	120 	0 	120 	35 	150
🐎Horseman 	100 	20 	0 	9 	125
♞Knight 	140 	0 	100 	24 	230
🐏Battering ram 	0 	300 	0 	200 	700
🎯Springald 	0 	250 	250 	30 	200

In [2]:
#!python -m pip install --upgrade --user -q ortools

In [26]:
# Import OR-Tools' wrapper for linear solvers
from ortools.linear_solver import pywraplp

#create the linear solver using the CBC backend
solver = pywraplp.Solver('Maximize army power',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

### Data

In [27]:
UNITS = [
    '🗡️Swordsmen',
    '🛡️Men-at-arms',
    '🏹Bowmen',
    '❌Crossbowmen',
    '🔫Handcannoneers',
    '🐎Horsemen',
    '♞Knights',
    '🐏Battering rams',
    '🎯Springalds',
    '🪨Mangonels',
]

DATA = [
    [60, 20, 0, 6, 70],
    [100, 0, 20, 12, 155],
    [30, 50, 0, 5, 70],
    [80, 0, 40, 12, 80],
    [120, 0, 120, 35, 150],
    [100, 20, 0, 9, 125],
    [140, 0, 100, 24, 230],
    [0, 300, 0, 200, 700],
    [0, 250, 250, 30, 200],
    [0, 400, 200, 12*3, 240]
]

RESOURCES = [183000, 90512, 80150]

#The goal is to minimize the resources that are used to get an army power > 1,000,000.

## 1. Create the variables we want to optimize

In [28]:
units = [solver.IntVar(0, solver.infinity(), unit) for unit in UNITS]

In [4]:
units

[🗡️Swordsmen,
 🛡️Men-at-arms,
 🏹Bowmen,
 ❌Crossbowmen,
 🔫Handcannoneers,
 🐎Horsemen,
 ♞Knights,
 🐏Battering rams,
 🎯Springalds,
 🪨Mangonels]

## 2. Add Constraints

In [29]:
# Resources constraint. sum cannot exceed resource limit
for r, _ in enumerate(RESOURCES):
    solver.Add(sum(DATA[u][r] * units[u] for u, _ in enumerate(units)) <= RESOURCES[r])

In [30]:
# Power constraint. higher than enemy
# for r, _ in enumerate(RESOURCES):
#     solver.Add(sum((DATA[u][-2] * 10 + DATA[u][-1]) * units[u] for u, _ in enumerate(units)) >= 1000001)

solver.Add(sum((DATA[u][-2] * 10 + DATA[u][-1]) * units[u] for u, _ in enumerate(units)) >= 1000001)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x10cb97ea0> >

## 3. Define Objective Function
### here is to minimize the resource needed to achieve power 1,000,000 to defeat enemy

In [31]:
# Single Minimize Call with Combined Costs
solver.Minimize(sum((DATA[u][0] + DATA[u][1] + DATA[u][2]) * units[u] for u, _ in enumerate(units)))

# # Multiple Minimize Calls Inside a Loop. but Linear Programming only allow for 1 objective function. only the last one will be considered in the following case.
# for r,_ in enumerate(RESOURCES):
#     solver.Minimize(sum(DATA[u][r] * units[u] for u,_ in enumerate(units)))

## 4. Solve the problem

In [32]:
status = solver.Solve()

## if an optimal solution has been found, print results

In [33]:
if status == pywraplp.Solver.OPTIMAL:
    print('================= Solution =================')
    print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
    print()

    power = sum((10 * DATA[u][-2] + DATA[u][-1]) * units[u].solution_value() for u, _ in enumerate(units))
    print(f'Optimal value = {solver.Objective().Value()} 🌾🪵🪙resources')
    print(f'Power = 💪{power}')
    print('Army:')
    for u, _ in enumerate(units):
        print(f' - {units[u].name()} = {units[u].solution_value()}')
    print()

    food = sum((DATA[u][0]) * units[u].solution_value() for u, _ in enumerate(units))
    wood = sum((DATA[u][1]) * units[u].solution_value() for u, _ in enumerate(units))
    gold = sum((DATA[u][2]) * units[u].solution_value() for u, _ in enumerate(units))
    print('Resources:')
    print(f' - 🌾Food = {food}')
    print(f' - 🪵Wood = {wood}')
    print(f' - 🪙Gold = {gold}')
else:
    print('The solver could not find an optimal solution.')


================= Solution =================
Solved in 22278.00 milliseconds in 1 iterations

Optimal value = 172100.0 🌾🪵🪙resources
Power = 💪1000105.0
Army:
 - 🗡️Swordsmen = 1.0
 - 🛡️Men-at-arms = 681.0
 - 🏹Bowmen = 0.0
 - ❌Crossbowmen = 0.0
 - 🔫Handcannoneers = 0.0
 - 🐎Horsemen = 0.0
 - ♞Knights = 0.0
 - 🐏Battering rams = 301.0
 - 🎯Springalds = 0.0
 - 🪨Mangonels = 0.0

Resources:
 - 🌾Food = 68160.0
 - 🪵Wood = 90320.0
 - 🪙Gold = 13620.0


# Automate, make it into a function call

In [34]:
def solve_army(UNITS, DATA, RESOURCES):
    
    # Create the linear solver using the CBC backend
    solver = pywraplp.Solver('Minimize resource consumption', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    # 1. Create the variables we want to optimize
    units = [solver.IntVar(0, solver.infinity(), unit) for unit in UNITS]

    # 2. Add constraints for each resource
    solver.Add(sum((10 * DATA[u][-2] + DATA[u][-1]) * units[u] for u, _ in enumerate(units)) >= 1000001)

    # Old constraints for limited resources
    for r, _ in enumerate(RESOURCES):
        solver.Add(sum(DATA[u][r] * units[u] for u, _ in enumerate(units)) <= RESOURCES[r])

    # 3. Minimize the objective function
    solver.Minimize(sum((DATA[u][0] + DATA[u][1] + DATA[u][2]) * units[u] for u, _ in enumerate(units)))

    # Solve problem
    status = solver.Solve()

    # If an optimal solution has been found, print results
    if status == pywraplp.Solver.OPTIMAL:
        print('================= Solution =================')
        print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
        print()

        power = sum((10 * DATA[u][-2] + DATA[u][-1]) * units[u].solution_value() for u, _ in enumerate(units))
        print(f'Optimal value = {solver.Objective().Value()} 🌾🪵🪙resources')
        print(f'Power = 💪{power}')
        print('Army:')
        for u, _ in enumerate(units):
            print(f' - {units[u].name()} = {units[u].solution_value()}')
        print()

        food = sum((DATA[u][0]) * units[u].solution_value() for u, _ in enumerate(units))
        wood = sum((DATA[u][1]) * units[u].solution_value() for u, _ in enumerate(units))
        gold = sum((DATA[u][2]) * units[u].solution_value() for u, _ in enumerate(units))
        print('Resources:')
        print(f' - 🌾Food = {food}')
        print(f' - 🪵Wood = {wood}')
        print(f' - 🪙Gold = {gold}')
    else:
        print('The solver could not find an optimal solution.')

solve_army(UNITS, DATA, RESOURCES)

================= Solution =================
Solved in 23.00 milliseconds in 1 iterations

Optimal value = 172100.0 🌾🪵🪙resources
Power = 💪1000105.0
Army:
 - 🗡️Swordsmen = 1.0
 - 🛡️Men-at-arms = 681.0
 - 🏹Bowmen = 0.0
 - ❌Crossbowmen = 0.0
 - 🔫Handcannoneers = 0.0
 - 🐎Horsemen = 0.0
 - ♞Knights = 0.0
 - 🐏Battering rams = 301.0
 - 🎯Springalds = 0.0
 - 🪨Mangonels = 0.0

Resources:
 - 🌾Food = 68160.0
 - 🪵Wood = 90320.0
 - 🪙Gold = 13620.0
